# Описание

Ниже - пример получения данных из источника, сохранение в S3 в parquet-файлы с последующим чтением и трансформацией данных в Spark

Для примера используются данные по персональной статистике игроков за матч за первые 5 игр сезона. В результате формируются два датафрейма:
- skater_game_stats_df - персональная статистика полевых игроков
- goalie_game_stats_df - персональная статистика голкиперов

В целевом решении датафреймы пишутся в ClickHouse в соответствующие таблицы. В данном блокноте запись в БД не предпологается

# Импорты и объявление пользовательского класса

In [1]:
import io
import os
import requests
from datetime import datetime, timezone
from typing import Iterator

import boto3
import pandas as pd
from pyspark.sql import Column, DataFrame, SparkSession
from pyspark.sql.functions import (
    col,
    concat,
    concat_ws,
    date_trunc,
    explode,
    lit,
    pandas_udf,
    split,
    struct,
    to_utc_timestamp,
    transform,
)

from pyspark.sql.types import StringType
from unidecode import unidecode


BASE_URL = 'https://api-web.nhle.com'
GAME_BOXSCORE_ENDPOINT = '/v1/gamecenter/{game_id}/boxscore'
ROOT_PREFIX = 'api/nhl'
EXAMPLE_PREFIX = f'{ROOT_PREFIX}/example_played_games'
GAMES = tuple(range(2025020001, 2025020006))


class S3Manager:
    def __init__(self, *, endpoint_url: str, aws_access_key_id: str, aws_secret_access_key: str, bucket: str):
        self.s3_client = boto3.client(
            service_name='s3',
            endpoint_url=endpoint_url,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        self.bucket = bucket

    def save_to_parquet(self, *, df: pd.DataFrame, key: str):
        """
        Сохраняет датафрейм df в S3 в parquet-формате по указанному префиксу key
        """

        buffer = io.BytesIO()
        df.to_parquet(buffer, index=False)
        buffer.seek(0)

        self.s3_client.put_object(
            Bucket=self.bucket,
            Key=key,
            Body=buffer.getvalue()
        )

    def get_subprefix_last_segments(self, prefix: str, delimiter: str = '/') -> Iterator[str]:
        """
        Возвращает генератор с названиям "подпапок" по указанному prefix.
        Например, есть путь 'api/parent/child/file.txt'.
        Если в prefix передать 'api/parent/', то по очереди вернутся все child "подпапки"
        """

        prefix = prefix.rstrip('/') + '/'

        paginator = self.s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=self.bucket, Prefix=prefix, Delimiter=delimiter)

        for page in pages:
            for subprefix in page.get('CommonPrefixes', []):
                *_, subprefix_last_segment, _ = subprefix['Prefix'].split('/')
                yield subprefix_last_segment

s3_manager = S3Manager(
    endpoint_url='http://minio:9000',
    aws_access_key_id=os.getenv('MINIO_ROOT_USER'),
    aws_secret_access_key=os.getenv('MINIO_ROOT_PASSWORD'),
    bucket=os.getenv('MINIO_BUCKET_NAME')
)

# Сохранение данных в S3

## Вспомогательные функции

In [2]:
def send_get_request(url: str) -> dict:
    """
    Отправляет GET-запрос на переданный url для получения данных. В случае отсутствия
    в ответе HTTP-ошибки возвращает тело ответа
    """

    response = requests.get(url)
    response.raise_for_status()
    return response.json()


def get_current_utc_ts():
    return datetime.now(timezone.utc)


def download_game_boxscores():
    """
    Скачивает информацию о персональной статистике игроков за матч
    """

    print(f'#️⃣ Завершенных игр получено: {len(GAMES)}')

    for num, game_id in enumerate(GAMES, 1):
        print(f'📥 {num}. Начинается скачивание данных игры {game_id}')

        data = send_get_request(BASE_URL + GAME_BOXSCORE_ENDPOINT.format(game_id=game_id))
        df = pd.json_normalize(data)

        if not df.empty:
            df['updated_at'] = get_current_utc_ts().isoformat()
            s3_manager.save_to_parquet(df=df, key=f'{EXAMPLE_PREFIX}/{game_id}/boxscore.parquet')
            print(f'✔️ Скачивание данных игры {game_id} завершено')
        else:
            print(f'⚠️ По игре {game_id} не получилось нормализовать данные')

## Запуск сохранения данных в S3

In [3]:
download_game_boxscores()

#️⃣ Завершенных игр получено: 5
📥 1. Начинается скачивание данных игры 2025020001
✔️ Скачивание данных игры 2025020001 завершено
📥 2. Начинается скачивание данных игры 2025020002
✔️ Скачивание данных игры 2025020002 завершено
📥 3. Начинается скачивание данных игры 2025020003
✔️ Скачивание данных игры 2025020003 завершено
📥 4. Начинается скачивание данных игры 2025020004
✔️ Скачивание данных игры 2025020004 завершено
📥 5. Начинается скачивание данных игры 2025020005
✔️ Скачивание данных игры 2025020005 завершено


# Чтение из S3 и трансформация данных в Spark

## Вспомогательные функции

In [4]:
def calculate_seconds_on_ice(time_on_ice: Column) -> Column:
    """
    Принимает колонку с игровым временем игрока за матч, представленным в виде строки.
    Возвращает колонку с игровым временм игрока за матч в секундах
    """
    time_parts = split(time_on_ice, ':')
    minutes = time_parts.getItem(0).cast('int')
    seconds = time_parts.getItem(1).cast('int')
    return minutes * 60 + seconds


def normalize_skater_game_stats_struct(x):
    """
    Принимает структуру и создает новую структуру только с необходимыми полями.
    Используется при трансформации матчевой статистики полевых игроков
    """

    return struct(
        x.playerId.alias('player_id'),
        x['name']['default'].alias('player_name'),
        x.position,
        x.sweaterNumber.alias('sweater_number'),
        x.goals,
        x.assists,
        x.points,
        x.plusMinus.alias('plus_minus'),
        x.pim.alias('penalty_minutes'),
        x.powerPlayGoals.alias('power_play_goals'),
        x.sog.alias('shots'),
        x.hits,
        x.blockedShots.alias('blocked_shots'),
        x.faceoffWinningPctg.alias('faceoff_winning_pctg'),
        x.giveaways,
        x.takeaways,
        x.shifts,
        x.toi.alias('time_on_ice')
    )


def normalize_goalie_game_stats_struct(x):
    """
    Принимает структуру и создает новую структуру только с необходимыми полями.
    Используется при трансформации матчевой статистики голкиперов
    """

    return struct(
        x.playerId.alias('player_id'),
        x['name']['default'].alias('player_name'),
        x.position,
        x.sweaterNumber.alias('sweater_number'),
        x.starter,
        x.decision,
        x.saves,
        x.shotsAgainst.alias('shots_against'),
        x.savePctg.alias('save_pctg'),
        x.goalsAgainst.alias('goals_against'),
        x.pim.alias('penalty_minutes'),
        x.toi.alias('time_on_ice')
    )

@pandas_udf(StringType())
def unidecode_udf(column: pd.Series) -> pd.Series:
    """
    Декодирует языковые unicode-символы в ASCII-символы.
    Используется, чтобы заменить буквы с различными знаками над ними
    ASCII-аналогами в названиях команд, именах и городах рождения хоккеистов
    Например:
        Olli Määttä → Olli Maatta
        Montréal Canadiens → Montreal Canadiens
    """
    return column.apply(lambda x: None if x is None or pd.isna(x) else unidecode(x))

## Инициализация SparkSession

In [5]:
spark = (
    SparkSession
    .builder
    .master('local')
    .appName('ExampleBoxscore')
    .config(
        'spark.jars.packages',
        'org.apache.hadoop:hadoop-aws:3.3.4,'
        'com.amazonaws:aws-java-sdk-bundle:1.12.262,'
        'com.clickhouse:clickhouse-jdbc:0.8.2'
    )
    .getOrCreate()
)

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set('fs.s3a.endpoint', 'http://minio:9000')
hadoop_conf.set('fs.s3a.access.key', os.getenv("MINIO_ROOT_USER"))
hadoop_conf.set('fs.s3a.secret.key', os.getenv("MINIO_ROOT_PASSWORD"))
hadoop_conf.set('fs.s3a.connection.ssl.enabled', 'false')
hadoop_conf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
hadoop_conf.set('fs.s3a.path.style.access', 'true')

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b301e78-edac-4804-92fe-88296d683b3c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.clickhouse#clickhouse-jdbc;0.8.2 in central
	found com.clickhouse#clickhouse-client;0.8.2 in central
	found com.clickhouse#clickhouse-data;0.8.2 in central
	found com.clickhouse#clickhouse-http-client;0.8.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.ap

## Чтение сырых данных

In [12]:
paths = (
    f's3a://{os.getenv("MINIO_BUCKET_NAME")}/{EXAMPLE_PREFIX}/{game_id}/boxscore.parquet'
    for game_id
    in GAMES
)

raw_boxscores_df = (
    spark
    .read
    .parquet(*paths)
    .cache()
)

print('👇 Сырые данные. Вся информация по одной игре в одной строке')
raw_boxscores_df.show(1, truncate=False, vertical=True)

25/12/25 10:43:17 WARN CacheManager: Asked to cache already cached data.


👇 Сырые данные. Вся информация по одной игре в одной строке
-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Трансформация сырых данных

In [13]:
raw_boxscores_df = (
    raw_boxscores_df
    .withColumnsRenamed({
        'homeTeam.id': 'home_team_id',
        'homeTeam.placeName.default': 'home_team_placename',
        'homeTeam.commonName.default': 'home_team_name',
        'homeTeam.abbrev': 'home_team_abbrev',
        'awayTeam.id': 'away_team_id',
        'awayTeam.placeName.default': 'away_team_placename',
        'awayTeam.commonName.default': 'away_team_name',
        'awayTeam.abbrev': 'away_team_abbrev',
        'playerByGameStats.homeTeam.forwards': 'home_team_forwards_stats',
        'playerByGameStats.homeTeam.defense': 'home_team_defense_stats',
        'playerByGameStats.homeTeam.goalies': 'home_team_goalies_stats',
        'playerByGameStats.awayTeam.forwards': 'away_team_forwards_stats',
        'playerByGameStats.awayTeam.defense': 'away_team_defense_stats',
        'playerByGameStats.awayTeam.goalies': 'away_team_goalies_stats'
    })
    .select(
        col('id').alias('game_id'),
        'season',
        date_trunc('second', to_utc_timestamp('startTimeUTC', 'UTC')).alias('start_time'),
        'home_team_id',
        'home_team_abbrev',
        concat_ws(' ', 'home_team_placename', 'home_team_name').alias('home_team_name'),
        'away_team_id',
        'away_team_abbrev',
        concat_ws(' ', 'away_team_placename', 'away_team_name').alias('away_team_name'),
        'home_team_forwards_stats',
        'home_team_defense_stats',
        'home_team_goalies_stats',
        'away_team_forwards_stats',
        'away_team_defense_stats',
        'away_team_goalies_stats',
        date_trunc('second', to_utc_timestamp('updated_at', 'UTC')).alias('updated_at')
    )
)

home_team_df = (
    raw_boxscores_df
    .select(
        'game_id',
        'season',
        'start_time',
        lit('home').alias('team_type'),
        col('home_team_id').alias('team_id'),
        col('home_team_abbrev').alias('team_abbrev'),
        col('home_team_name').alias('team_name'),
        concat(
            transform('home_team_forwards_stats', lambda x: normalize_skater_game_stats_struct(x)),
            transform('home_team_defense_stats', lambda x: normalize_skater_game_stats_struct(x)),
        ).alias('skaters_stats'),
        transform(
            'home_team_goalies_stats', lambda x: normalize_goalie_game_stats_struct(x)
        ).alias('goalies_stats'),
        'updated_at'
    )
)

away_team_df = (
    raw_boxscores_df
    .select(
        'game_id',
        'season',
        'start_time',
        lit('away').alias('team_type'),
        col('away_team_id').alias('team_id'),
        col('away_team_abbrev').alias('team_abbrev'),
        col('away_team_name').alias('team_name'),
        concat(
            transform('away_team_forwards_stats', lambda x: normalize_skater_game_stats_struct(x)),
            transform('away_team_defense_stats', lambda x: normalize_skater_game_stats_struct(x)),
        ).alias('skaters_stats'),
        transform(
            'away_team_goalies_stats', lambda x: normalize_goalie_game_stats_struct(x)
        ).alias('goalies_stats'),
        'updated_at'
    )
)

separated_by_teams_df = home_team_df.unionByName(away_team_df).cache()

print('👇 Данные по одной игре разделены на команду-хозяев и команду-гостей')
separated_by_teams_df.orderBy('game_id').show(2, truncate=False, vertical=True)
separated_by_teams_df.printSchema()

25/12/25 10:43:21 WARN CacheManager: Asked to cache already cached data.


👇 Данные по одной игре разделены на команду-хозяев и команду-гостей
-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Формирование датафрейма со статистикой полевых игроков

In [18]:
skater_game_stats_df = (
        separated_by_teams_df
        .where(col('game_id').isin(sorted(GAMES)))
        .withColumn(
            'skater_stats',
            explode('skaters_stats')
        )
        .select(
            'game_id',
            'season',
            'start_time',
            'team_type',
            'team_id',
            'team_abbrev',
            unidecode_udf(col('team_name')).alias('team_name'),
            'skater_stats.player_id',
            unidecode_udf(col('skater_stats.player_name')).alias('player_name'),
            'skater_stats.position',
            'skater_stats.sweater_number',
            'skater_stats.goals',
            'skater_stats.assists',
            'skater_stats.points',
            'skater_stats.plus_minus',
            'skater_stats.penalty_minutes',
            'skater_stats.power_play_goals',
            'skater_stats.shots',
            'skater_stats.hits',
            'skater_stats.blocked_shots',
            'skater_stats.faceoff_winning_pctg',
            'skater_stats.giveaways',
            'skater_stats.takeaways',
            'skater_stats.shifts',
            'skater_stats.time_on_ice',
            calculate_seconds_on_ice(col('skater_stats.time_on_ice')).alias('time_on_ice_seconds'),
            'updated_at'
        )
    )

print('👇 Для каждого полевого игрока создана отдельная строка')
skater_game_stats_df.show(10, truncate=False)
skater_game_stats_df.printSchema()

👇 Для каждого полевого игрока создана отдельная строка
+----------+--------+-------------------+---------+-------+-----------+----------------+---------+--------------+--------+--------------+-----+-------+------+----------+---------------+----------------+-----+----+-------------+--------------------+---------+---------+------+-----------+-------------------+-------------------+
|game_id   |season  |start_time         |team_type|team_id|team_abbrev|team_name       |player_id|player_name   |position|sweater_number|goals|assists|points|plus_minus|penalty_minutes|power_play_goals|shots|hits|blocked_shots|faceoff_winning_pctg|giveaways|takeaways|shifts|time_on_ice|time_on_ice_seconds|updated_at         |
+----------+--------+-------------------+---------+-------+-----------+----------------+---------+--------------+--------+--------------+-----+-------+------+----------+---------------+----------------+-----+----+-------------+--------------------+---------+---------+------+-----------+--

## Формирование датафрейма со статистикой голкиперов

In [17]:
goalie_game_stats_df = (
        separated_by_teams_df
        .where(col('game_id').isin(sorted(GAMES)))
        .withColumn(
            'goalie_stats',
            explode('goalies_stats')
        )
        .select(
            'game_id',
            'season',
            'start_time',
            'team_type',
            'team_id',
            'team_abbrev',
            unidecode_udf(col('team_name')).alias('team_name'),
            'goalie_stats.player_id',
            unidecode_udf(col('goalie_stats.player_name')).alias('player_name'),
            'goalie_stats.position',
            'goalie_stats.sweater_number',
            'goalie_stats.starter',
            'goalie_stats.decision',
            'goalie_stats.saves',
            'goalie_stats.shots_against',
            'goalie_stats.save_pctg',
            'goalie_stats.goals_against',
            'goalie_stats.penalty_minutes',
            'goalie_stats.time_on_ice',
            calculate_seconds_on_ice(col('goalie_stats.time_on_ice')).alias('time_on_ice_seconds'),
            'updated_at'
        )
    )

print('👇 Для каждого голкипера создана отдельная строка')
goalie_game_stats_df.show(10, truncate=False)
goalie_game_stats_df.printSchema()

👇 Для каждого голкипера создана отдельная строка


+----------+--------+-------------------+---------+-------+-----------+-------------------+---------+-------------+--------+--------------+-------+--------+-----+-------------+---------+-------------+---------------+-----------+-------------------+-------------------+
|game_id   |season  |start_time         |team_type|team_id|team_abbrev|team_name          |player_id|player_name  |position|sweater_number|starter|decision|saves|shots_against|save_pctg|goals_against|penalty_minutes|time_on_ice|time_on_ice_seconds|updated_at         |
+----------+--------+-------------------+---------+-------+-----------+-------------------+---------+-------------+--------+--------------+-------+--------+-----+-------------+---------+-------------+---------------+-----------+-------------------+-------------------+
|2025020001|20252026|2025-10-07 21:00:00|home     |13     |FLA        |Florida Panthers   |8480193  |D. Tarasov   |G       |40            |false  |NULL    |0    |0            |NULL     |0      

# Остановка сессии

In [19]:
spark.stop()